In [1]:
import re
import pandas as pd
import re

# Get Tourneys
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [6]:
# Get Games
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney']=='(nc)' or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']=='' or gameDict['arena']=='Boston Garden' or gameDict['arena']=='VW Arena'):
            gameDict['location']='Neutral'
        if((gameDict['arena']=='Gutterson' and gameDict['opponent']=='Vermont') or (gameDict['arena']=='Houston' and gameDict['opponent']=='Rensselaer') or (gameDict['arena']=='Broadmoor' and gameDict['opponent']=='Colorado College') or (gameDict['arena']=='DEC Center' and gameDict['opponent']=='Minnesota Duluth')or (gameDict['arena']=='Magness Arena' and gameDict['opponent']=='Denver')or (gameDict['arena']=='Mariucci Arena' and gameDict['opponent']=='Minnesota')or (gameDict['arena']=='Munn Ice Arena' and gameDict['opponent']=='Michigan State')or (gameDict['arena']=='Walker Arena' and gameDict['opponent']=='Clarkson')or (gameDict['arena']=='Thompson Arena' and gameDict['opponent']=='Dartmouth')or (gameDict['arena']=='St. Louis Arena' and gameDict['opponent']=='St. Louis') or (gameDict['arena']=='Sullivan Arena' and gameDict['opponent']=='Alaska Anchorage')):
            gameDict['location']='Away'

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        if(int(gameDict['month']) >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(int(gameDict['month']) <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        if(gameDict['season']=='1973-74' and gameDict['date']=='12/12/1973'):
            coach='Jack Parker'
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameList.append(gameDict)
f.close()
dfGames=pd.DataFrame(gameList)

In [10]:
for i in dfGames.loc[dfGames['date'].between('9/1/2012','5/1/2016')].groupby('result').size()[['W','L','T']]:
    print(i,end='-')
print('\b')

80-58-16-


In [14]:
(80+(16/2))/(80+58+16)

0.5714285714285714

In [11]:
dfGames.loc[(dfGames['date'].between('9/1/2012','5/1/2016')) & (dfGames['gameType']=='Exhibition') ]

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,captain,conference,season,note,month,day,year,BUScore,OppoScore,GD
2426,2012-10-07,E,None,Agganis Arena,Toronto,Exhibition,None,5-0,Home,Jack Parker,Wade Megan,Hockey East,2012-13,,10,7,2012,5,0,5
2466,2013-10-05,E,None,Agganis Arena,St. Francis Xavier,Exhibition,None,2-1,Home,David Quinn,"Patrick MacGregor, Garrett Noonan",Hockey East,2013-14,,10,5,2013,2,1,1
2502,2014-10-04,E,None,Agganis Arena,St. ThomasN.B.,Exhibition,None,12-1,Home,David Quinn,Matt Grzelcyk,Hockey East,2014-15,,10,4,2014,12,1,11
2504,2014-10-18,E,None,Agganis Arena,US Under-18 Team,Exhibition,None,6-4,Home,David Quinn,Matt Grzelcyk,Hockey East,2014-15,,10,18,2014,6,4,2
2520,2014-12-19,E,None,Brown Arena,US Under-18 Team,Exhibition,None,2-5,Home,David Quinn,Matt Grzelcyk,Hockey East,2014-15,,12,19,2014,2,5,3
2546,2015-10-03,E,None,Agganis Arena,Acadia,Exhibition,None,4-2,Home,David Quinn,Matt Grzelcyk,Hockey East,2015-16,,10,3,2015,4,2,2
2548,2015-10-16,E,None,Agganis Arena,US Under-18 Team,Exhibition,None,4-7,Home,David Quinn,Matt Grzelcyk,Hockey East,2015-16,,10,16,2015,4,7,3


In [12]:
for i in dfGames.loc[dfGames['date'].between('5/1/2016','5/1/2022')].groupby('result').size()[['W','L','T']]:
    print(i,end='-')
print('\b')

104-75-23-


In [13]:
(104+(23/2))/(104+75+23)

0.5717821782178217

In [28]:
# Get Jerseys
fileName=('JerseyNumbers.txt')
tourneys=[]
teamSet=set()
playerList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    numDict={'number':0,'player':'','season':''}
    for i in rows:
        
        numSearch=re.search("#(.*)",i)
        if numSearch != None:
            number=numSearch.group(1)
        if("Retired - " in i):
                continue
        playerSearch=re.search("(\d*-\d*) (.*)",i)
        if playerSearch != None:
            season=playerSearch.group(1)
            numDict={'number':int(number),
                 'season':convertSeasons(season),
                 'name':playerSearch.group(2)}
            playerList.append(numDict)
f.close()
dfJersey=pd.DataFrame(playerList)

In [5]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=None
        return val

In [6]:
def convertToFloat(val):
        try:
            val=float(val)
        except:
            val=float('nan')
        return val

In [7]:
def convertSeasons(season):
        gap=season.split(',')
        years=season[2:].split('-')
        seasonStr=''
        if(len(gap)>1):
            for i in gap:
                seasonStr+=convertSeasons(i)
        else:
            yearDiff=abs(int(years[0])-int(years[1]))
            if(yearDiff>6):
                yearDiff=100-yearDiff
            firstHalf=season[:4]
            seasonStr=''
            for i in range(yearDiff):
                secondHalf=int(firstHalf)+1
                seasonStr+=str(firstHalf)+'-'+str(secondHalf)[2:]+','
                firstHalf=secondHalf
        return seasonStr[:-1]

In [10]:
fileName=('SkaterStats.txt')
tourneys=[]
teamSet=set()
skateList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        skaterSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)\/(\S*)',i)
        if skaterSearch!=None:
            skaterDict={'last':skaterSearch.group(1),
                       'first':skaterSearch.group(2),
                        'name':skaterSearch.group(2)+' '+skaterSearch.group(1),
                       'seasons':convertSeasons(skaterSearch.group(3)),
                       'gp':convertToInt(skaterSearch.group(4)),
                       'goals':convertToInt(skaterSearch.group(5)),
                       'assists':convertToInt(skaterSearch.group(6)),
                       'pts':convertToInt(skaterSearch.group(7)),
                       'pen':convertToInt(skaterSearch.group(8)),
                       'pim':convertToInt(skaterSearch.group(9))}
            skateList.append(skaterDict)
dfSkate=pd.DataFrame(skateList)

In [9]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)            

In [38]:
dfGames

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,captain,conference,season,note,month,day,year,BUScore,OppoScore,GD
0,2/6/1918,L,None,Boston Arena,Boston College,Non-Conference,None,1-3,Home,Edgar Burkhardt,Nelson Jost,Independent,1917-18,,2,6,1918,1,3,2
1,1/16/1920,L,None,Ice Pavilion,Mass. Agricultural,Non-Conference,None,2-10,Away,Harold Stuart,Edward Hersey,Independent,1919-20,,1,16,1920,2,10,8
2,2/4/1920,L,None,University Rink,Boston College,Non-Conference,None,0-9,Away,Harold Stuart,Edward Hersey,Independent,1919-20,,2,4,1920,0,9,9
3,12/8/1922,L,None,Boston Arena,MIT,Non-Conference,None,4-7,Home,John O'Hare,Edward Hersey,Independent,1922-23,,12,8,1922,4,7,3
4,12/13/1922,L,None,Boston Arena,Harvard,Non-Conference,None,0-2,Home,John O'Hare,Edward Hersey,Independent,1922-23,,12,13,1922,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2793,2/26/2022,W,None,Agganis Arena,Boston College,Conference,None,6-3,Home,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,2,26,2022,6,3,3
2794,2/27/2022,L,None,Conte Forum,Boston College,Conference,None,1-3,Away,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,2,27,2022,1,3,2
2795,3/4/2022,W,None,Alfond Arena,Maine,Conference,None,5-1,Away,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,3,4,2022,5,1,4
2796,3/5/2022,L,None,Alfond Arena,Maine,Conference,None,1-8,Away,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,3,5,2022,1,8,7


In [39]:
dfSkate.loc[dfSkate['name'].isin((dfJersey.loc[dfJersey['number']==3]['name']))].sort_values('gp',ascending=False)

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
641,Schaeffer,Kevin,Kevin Schaeffer,"2003-04,2004-05,2005-06,2006-07",152.0,17,37,54,39.0,86.0
540,Oksanen,Ahti,Ahti Oksanen,"2012-13,2013-14,2014-15,2015-16",150.0,51,64,115,30.0,68.0
538,Octeau,Jay,Jay Octeau,"1983-84,1984-85,1985-86,1986-87",143.0,16,70,86,73.0,149.0
647,Shattenkirk,Kevin,Kevin Shattenkirk,"2007-08,2008-09,2009-10",121.0,18,60,78,58.0,116.0
7,Amonte,Ty,Ty Amonte,"2017-18,2018-19,2019-20,2020-21,2021-22",119.0,17,20,37,32.0,91.0
134,Cotter,Bill,Bill Cotter,"1977-78,1978-79,1979-80,1980-81",117.0,30,54,84,54.0,143.0
547,O’Sullivan,Chris,Chris O’Sullivan,"1992-93,1993-94,1994-95,1995-96",113.0,40,88,128,50.0,124.0
741,Whitney,Ryan,Ryan Whitney,"2001-02,2002-03,2003-04",107.0,16,43,59,60.0,152.0
201,Dunbar,Dale,Dale Dunbar,"1980-811982-83,1983-84,1984-8",104.0,5,41,46,73.0,157.0
226,Fay,Gary,Gary Fay,"1973-74,1974-75,1975-76,1976-77",101.0,13,89,102,19.0,46.0
